In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tqdm import trange

In [ ]:
data_image  = np.load('data/images_128.npy')
data_keys   = np.load('data/keys.npy')
data_memory = np.load('data/memory.npy')

In [ ]:
train_size = 9000
train_X = data_image[0:train_size]
train_Y = data_keys[0:train_size]
valid_X = data_image[train_size:]
valid_Y = data_keys[train_size:]

In [ ]:
print(data_image.shape, data_image.dtype, data_image.min(), data_image.max())
print(data_keys.shape, data_keys.dtype, data_keys.min(), data_keys.max())
print(np.sum(data_keys, axis=0) / np.sum(data_keys))

In [ ]:
sets = []
for unique in np.unique(data_keys, axis=0):
    sets.append([])
i = 0
for unique in np.unique(data_keys, axis=0):
    for z in data_keys:
        if (z == unique).all():
            sets[i].append(z)
    i+=1
i = 0
for unique in np.unique(data_keys, axis=0):
    print(unique, len(sets[i]))
    i+=1

In [ ]:
weight_up = 0.25
weight_direction = 1
weight_down = 1.5
class_weight = {0: weight_up, 1: weight_direction, 2: weight_down, 3: weight_direction}

In [ ]:
# model = tf.keras.applications.MobileNetV3Small(
model = tf.keras.applications.MobileNetV2(
    input_shape=(128, 128, 3),
    weights=None,
    classes=4,
    classifier_activation="sigmoid")
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=[tf.keras.metrics.MeanSquaredError()])

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='model_alpha_best',
    save_weights_only=False,
    save_best_only=True)

In [ ]:
model.fit(train_X, train_Y, shuffle=True,
          validation_data=(valid_X, valid_Y),
          epochs=100, batch_size=32, class_weight=class_weight,
          verbose=2,
         callbacks=[model_checkpoint_callback])

In [ ]:
model = tf.keras.models.load_model('model_alpha_best')

In [ ]:
bs = 100
predict = np.zeros(valid_Y.shape, dtype=data_keys.dtype)
for z in trange(0, valid_Y.shape[0], bs):
    y = ((model.predict(valid_X[z:z+bs])>0.5)[0:bs]*1.0)
    predict[z:z+bs] = y

In [ ]:
print(np.sum(valid_Y, axis=0) / np.sum(valid_Y))
print(np.sum(predict, axis=0) / np.sum(predict))